In [1]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def request(url):
    # response是一个Python requests库中的Response对象，它包含了服务器响应的所有信息
    response = requests.get(url)
    remessage={
        "status":response.status_code,
        "headers":response.headers,
        "content":response.content,
    }

    with open('sneakerNowre.html', 'w') as f:
        f.write(response.content.decode('utf-8'))
    return remessage

In [3]:
def parseSneaker(url,MaxSize):
    dicts={}
    title=[]
    link=[]
    datelist=[]
    texts=[]
    sources=[]
    images=[]
    sourcelink=[]
    for i in range(1,MaxSize+1):
        if i==1:
            remessage=request(url)
        else:
            remessage=request(url+f"/page/{i}/")
        content=remessage["content"]
        soup = BeautifulSoup(content, 'html.parser')
        tags = soup.find_all('div', {'class': 'nowre-list-item-info'})
        for tag in tags:
            child_tags = tag.find_all('div', {'class': 'nowre-list-item-title'})
            dates=tag.find_all('span', {'class': 'item-time'})
            for child_tag in child_tags:
                #print(child_tag.text.strip())
                cchild_tags = child_tag.find('a')["href"]
                # print(cchild_tags)
                title.append(child_tag.text.strip())
                link.append(cchild_tags)
            for datee in dates:
                datelist.append(datee.text)
    for url in link:
        response2 = requests.get(url)
        childmessage=response2.content
        soup = BeautifulSoup(childmessage, 'html.parser')
        p_tags = soup.find_all('p')
        text=""
        for p in p_tags:
            text=text+p.text
        texts.append(text)
        source=soup.find_all('div', {'class': 'info-common-source text-gray'})
        if source!=[]:
            href=source[0].find("a")["href"]
            a=source[0].find("a")
            sourcelink.append(href)
            sources.append(a.text)
        else:
            sourcelink.append("None")
            sources.append("None")
        image=soup.find_all('img',{'class':'img-responsive center-table'})
        imagee=[]
        for i in image:
            src=i['src']
            imagee.append(src)
        images.append(imagee)
    data=pd.DataFrame({
        "title":title,
        "link":link,
        "date":datelist,
        "content":texts,
        "image":images,
        "source":sources,
        "sourcelink":sourcelink
        })
    data.to_csv('NOWREdemo.csv') # 保存为csv文件
    return dicts

In [5]:
# url的组合
base_url="https://nowre.com"
expand=["/news","/topic","/fashion","/sneaker","/life","/tech"]
link=[]
for ele in expand:
    link.append(base_url+ele)

url = link[3]
MaxSize=1648


